In [1]:
import os

# Why Torch? You' ll find the answer in the .md files! 
os.environ["KERAS_BACKEND"] = "torch"

In [4]:
# Data loading
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

# Image size? todo

full_train_dataset = DataLoader(datasets.ImageFolder("../data/train/", transform=transforms.Compose([
    transforms.Resize((128, 128)), transforms.ToTensor(),
])))

validation_dataset = DataLoader(datasets.ImageFolder("../data/test/", transform=transforms.Compose([
    transforms.Resize((128, 128)), transforms.ToTensor()
])))

# K-FOLD Loading
We load data and check if functions work as intended

In [3]:
from k_fold_cv.k_fold_generator import KFoldController
FOLDS = 5

k_fold_controller = KFoldController(FOLDS)
k_fold_controller.load_data(full_train_dataset)

for i in range(FOLDS):
    train, val = k_fold_controller.get_data_for_fold(i)

(<torch.utils.data.dataset.ConcatDataset object at 0x7fcca47f2810>, <torch.utils.data.dataset.Subset object at 0x7fcca45750d0>)


In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision import transforms
import matplotlib.pyplot as plt

train_dataloader = DataLoader(datasets.ImageFolder("../data/train/", transform=transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])), batch_size=32, shuffle=True)

validation_dataloader = DataLoader(
    datasets.ImageFolder("../data/test/",
                         transform=transforms.Compose([transforms.Resize((128, 128)), transforms.ToTensor()])),
    batch_size=32, shuffle=True
)

train_features, train_labels = next(iter(train_dataloader))

img = train_features[0].squeeze()
label = train_labels[0]
print(label)
plt.imshow(img.T, cmap="gray")
plt.show()

In [ ]:
from models.simple_neural_network import simple_neural_network

model = simple_neural_network((128, 128, 3), 2)
model.summary()

In [ ]:
model.fit(train_dataloader, epochs=150, validation_data=validation_dataloader)

In [ ]:
from keras.utils import plot_model

plot_model(model, show_shapes=True)